### Objective:
To build a hybrid recommendation engine that combines collaborative filtering and content-based filtering to suggest the most relevant products for customers, improving personalization, cross-selling, and customer retention.

In [26]:
#importing libraries

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import fpgrowth

In [2]:
df = pd.read_excel(r"C:\Users\ikeol\OneDrive\Desktop\Product Analytics (Projects)\Online Retail.xlsx")


In [30]:
df.head(20)

,InvoiceNo,CustomerID,StockCode,Description,Quantity
0,536365,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,17850.0,71053,WHITE METAL LANTERN,6
2,536365,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6
5,536365,17850.0,22752,SET 7 BABUSHKA NESTING BOXES,2
6,536365,17850.0,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6
7,536366,17850.0,22633,HAND WARMER UNION JACK,6
8,536366,17850.0,22632,HAND WARMER RED POLKA DOT,6
9,536367,13047.0,84879,ASSORTED COLOUR BIRD ORNAMENT,32


In [29]:
df = df[['InvoiceNo', 'CustomerID', 'StockCode', 'Description', 'Quantity']]

df.head(20)

,InvoiceNo,CustomerID,StockCode,Description,Quantity
0,536365,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,17850.0,71053,WHITE METAL LANTERN,6
2,536365,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6
5,536365,17850.0,22752,SET 7 BABUSHKA NESTING BOXES,2
6,536365,17850.0,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6
7,536366,17850.0,22633,HAND WARMER UNION JACK,6
8,536366,17850.0,22632,HAND WARMER RED POLKA DOT,6
9,536367,13047.0,84879,ASSORTED COLOUR BIRD ORNAMENT,32


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   CustomerID   406829 non-null  float64
 2   StockCode    541909 non-null  object 
 3   Description  540455 non-null  object 
 4   Quantity     541909 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 20.7+ MB


In [6]:
has_duplicates = df.duplicated().any()

num_duplicates = df.duplicated().sum()

print(has_duplicates)
print('The number of duplicate row is ' + str(num_duplicates))

True
The number of duplicate row is 5429


In [28]:
#Removing Duplicates from the dataframe
df = df.copy()

df.drop_duplicates(inplace = True)
df.head(20)

,InvoiceNo,CustomerID,StockCode,Description,Quantity
0,536365,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,17850.0,71053,WHITE METAL LANTERN,6
2,536365,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6
5,536365,17850.0,22752,SET 7 BABUSHKA NESTING BOXES,2
6,536365,17850.0,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6
7,536366,17850.0,22633,HAND WARMER UNION JACK,6
8,536366,17850.0,22632,HAND WARMER RED POLKA DOT,6
9,536367,13047.0,84879,ASSORTED COLOUR BIRD ORNAMENT,32


In [8]:
# Handling Missing Values
df['CustomerID'] = df['CustomerID'].replace('', np.nan)

In [9]:
# Dropping rows with missing product details
df.dropna(subset = ['Description'], inplace = True)

In [10]:
# Keeping only positive quantity values

df = df[df['Quantity'] > 0]

In [11]:
print("The dataset after cleaning has:" + str(df.shape))

The dataset after cleaning has:(525375, 5)


### Market Basket Analysis (Guest Users)

In [12]:
#Filtering guest users by missing customer_id

guest_users = df[df['CustomerID'].isna()]

In [13]:
# Creating a transaction-product matrix

basket = guest_users.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)

In [14]:
#Converting quantites into binary flags

basket = basket.loc[:, basket.sum() > 50]

basket = basket > 0

In [15]:
frequent_items = fpgrowth(basket, min_support=0.05, use_colnames=True)
frequent_items.head()

,support,itemsets
0,0.432500,(DOTCOM POSTAGE)
1,0.307500,(JUMBO BAG RED RETROSPOT)
2,0.213125,(RECYCLING BAG RETROSPOT )
3,0.204375,(RED TOADSTOOL LED NIGHT LIGHT)
4,0.203750,(SUKI SHOULDER BAG)


In [16]:
# Generating Association rules

rules = association_rules(frequent_items, metric="lift", min_threshold=1)

In [17]:
# Sorting rules by confidence

rules = rules.sort_values("confidence", ascending = False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
117916,"(JUMBO BAG RED RETROSPOT, SUKI SHOULDER BAG, ...",(DOTCOM POSTAGE),0.050625,0.4325,0.050625,1.0,2.312139,1.0,0.028730,inf,0.597762,0.117052,1.0,0.558526
71556,"(SUKI SHOULDER BAG, SCOTTIE DOGS BABY BIB)",(DOTCOM POSTAGE),0.064375,0.4325,0.064375,1.0,2.312139,1.0,0.036533,inf,0.606546,0.148844,1.0,0.574422
18179,"(NATURAL SLATE HEART CHALKBOARD , JUMBO BAG WO...",(DOTCOM POSTAGE),0.053750,0.4325,0.053750,1.0,2.312139,1.0,0.030503,inf,0.599736,0.124277,1.0,0.562139
71594,"(WOODLAND CHARLOTTE BAG, SCOTTIE DOGS BABY BIB)",(DOTCOM POSTAGE),0.060625,0.4325,0.060625,1.0,2.312139,1.0,0.034405,inf,0.604125,0.140173,1.0,0.570087
71588,"(VINTAGE PAISLEY STATIONERY SET, SCOTTIE DOGS ...",(DOTCOM POSTAGE),0.061875,0.4325,0.061875,1.0,2.312139,1.0,0.035114,inf,0.604930,0.143064,1.0,0.571532


### Collaborative Filtering (Registered Customers)

In [18]:
# filter registered users

registered_users = df[df['CustomerID'].notna()]

In [19]:
# Creating a customer and product matrix

customer_product_matrix = registered_users.pivot_table(index='CustomerID', columns='Description', values='Quantity', 
    aggfunc='sum'
).fillna(0)

In [20]:
# Computing the cosine similarity between customers

similarity = cosine_similarity(customer_product_matrix)
similarity_df = pd.DataFrame(similarity, 
                             index = customer_product_matrix.index, 
                             columns = customer_product_matrix.index)

In [21]:
# Creating a Function to recommend products based on similar customers

registered_users = df[df['CustomerID'].notna()]

customer_product_matrix = registered_users.pivot_table(index='CustomerID', columns='Description', values='Quantity', 
    aggfunc='sum'
).fillna(0)

similarity = cosine_similarity(customer_product_matrix)
similarity_df = pd.DataFrame(similarity, 
                             index = customer_product_matrix.index, 
                             columns = customer_product_matrix.index)

In [22]:
def recommend_products(CustomerID, num_recommendations=3):
    
    # Check if CustomerID exists in similarity_df
    if CustomerID not in similarity_df.index:
        return [f"CustomerID {CustomerID} not found in similarity matrix. Try another ID."]
    
    # Get similar users sorted by similarity score
    similar_users = similarity_df.loc[CustomerID].sort_values(ascending=False)
    
    # Remove the customer themselves from the list
    similar_users = similar_users.drop(CustomerID, errors="ignore")
    
    # Get the top similar customers
    top_similar_users = similar_users.index[:5]
    
    # Get all products bought by those top similar users
    similar_purchases = registered_users[registered_users['CustomerID'].isin(top_similar_users)]
    
    # Get products already bought by the target customer
    purchased = registered_users[registered_users['CustomerID'] == CustomerID]['Description']
    
    # Exclude already purchased items
    recommended_products = similar_purchases[~similar_purchases['Description'].isin(purchased)]
    
    # Get the most popular products among similar users
    top_recommendations = (
        recommended_products['Description']
        .value_counts()
        .head(num_recommendations)
        .index
        .tolist()
    )
    
    # If no recommendations found
    if not top_recommendations:
        return [f"No recommendations available for CustomerID {CustomerID}."]
    # Return Top Recommendations
    return top_recommendations


# Creating an input function so users can input CustomerID
try:
    # Input function
    customer_id = int(input("Enter a Customer ID to get product recommendations: "))

    # Get recommendations
    recommendations = recommend_products(customer_id, num_recommendations=3)


    print("\nRecommended Products:")
    for idx, product in enumerate(recommendations, start=1):
        print(f"{idx}. {product}")

except ValueError:
    print("Invalid input. Please enter a numeric Customer ID.")

Enter a Customer ID to get product recommendations: 17850

Recommended Products:
1. RED HANGING HEART T-LIGHT HOLDER
2. FAMILY ALBUM WHITE PICTURE FRAME
3. WOOD BLACK BOARD ANT WHITE FINISH


### Hybrid Recommendation

In [23]:
def hybrid_recommendation(customer_id, last_product=None):
    """
    Combines collaborative filtering + market basket recommendations.
    """
    if pd.isna(customer_id):
        # Guest user → Use Market Basket Analysis
        guest_recs = rules[rules['antecedents'].apply(lambda x: last_product in list(x))]
        if not guest_recs.empty:
            return list(guest_recs['consequents'].iloc[0])
        else:
            return ["No guest recommendations available"]
    else:
        # Registered customer → Use Collaborative Filtering
        return recommend_products(customer_id, num_recommendations=3)

# Example: Hybrid prediction
print(hybrid_recommendation(12557, last_product="DOTCOM POSTAGE"))  # Registered customer
print(hybrid_recommendation(np.nan, last_product="LUNCH BAG SPACEBOY DESIGN"))  # Guest user


['POSTAGE', 'LUNCH BAG SPACEBOY DESIGN ', 'CHILDRENS CUTLERY SPACEBOY ']
['No guest recommendations available']


In [27]:
# Creating the recommendations table for all unique CustomerID

final_df = df[['CustomerID', 'Description']].drop_duplicates(subset=['CustomerID'])
final_df['recommendations'] = final_df.apply(
    lambda x: hybrid_recommendation(x['CustomerID'], x['Description']), axis=1
)

# Display final recommendations
final_df.head()


,CustomerID,Description,recommendations
0,17850.0,WHITE HANGING HEART T-LIGHT HOLDER,"[RED HANGING HEART T-LIGHT HOLDER, FAMILY ALBU..."
9,13047.0,ASSORTED COLOUR BIRD ORNAMENT,"[DOORMAT FANCY FONT HOME SWEET HOME, DOORMAT K..."
26,12583.0,ALARM CLOCK BAKELIKE PINK,"[LUNCH BAG SPACEBOY DESIGN , LUNCH BAG DOLLY G..."
46,13748.0,PAPER CHAIN KIT 50'S CHRISTMAS,"[PAPER CHAIN KIT VINTAGE CHRISTMAS, 12 COLOURE..."
65,15100.0,VICTORIAN SEWING BOX LARGE,"[WOOD BLACK BOARD ANT WHITE FINISH, ENAMEL BRE..."


In [87]:
final_df.to_csv(r"C:\Users\ikeol\OneDrive\Desktop\Product Analytics (Projects)\customer_recommendations.csv", index=False)

final_df.head(15)

,CustomerID,Description,recommendations
0,17850.0,WHITE HANGING HEART T-LIGHT HOLDER,"[RED HANGING HEART T-LIGHT HOLDER, FAMILY ALBU..."
9,13047.0,ASSORTED COLOUR BIRD ORNAMENT,"[DOORMAT FANCY FONT HOME SWEET HOME, DOORMAT K..."
26,12583.0,ALARM CLOCK BAKELIKE PINK,"[LUNCH BAG SPACEBOY DESIGN , LUNCH BAG DOLLY G..."
46,13748.0,PAPER CHAIN KIT 50'S CHRISTMAS,"[PAPER CHAIN KIT VINTAGE CHRISTMAS, 12 COLOURE..."
65,15100.0,VICTORIAN SEWING BOX LARGE,"[WOOD BLACK BOARD ANT WHITE FINISH, ENAMEL BRE..."
82,15291.0,HOT WATER BOTTLE TEA AND SYMPATHY,"[RED TOADSTOOL LED NIGHT LIGHT, RED SPOT CERAM..."
86,14688.0,JUMBO BAG PINK POLKADOT,"[LUNCH BAG DOLLY GIRL DESIGN, JUMBO BAG VINTAG..."
105,17809.0,JAM MAKING SET PRINTED,"[POSTAGE, PAPER CHAIN KIT 50'S CHRISTMAS , HEA..."
106,15311.0,RETROSPOT TEA SET CERAMIC 11 PC,"[JUMBO BAG VINTAGE LEAF, JUMBO BAG PEARS, PART..."
142,16098.0,INFLATABLE POLITICAL GLOBE,"[VINTAGE HEADS AND TAILS CARD GAME , CHOCOLATE..."


In [25]:
#### Ignore 

recommendations_list = []

for customer_id in df['CustomerID'].unique():
    recs = recommend_products(customer_id, num_recommendations=3)
    past = df[df['CustomerID'] == customer_id]['Description'].unique().tolist()
    recommendations_list.append({
        "CustomerID": customer_id,
        "Past Purchases": ", ".join(past),
        "Recommended Products": ", ".join(recs)
    })

recommendations_df = pd.DataFrame(recommendations_list)

recommendations_df.to_excel("Ecommerce_Recommendations.xlsx", index=False)

print("Recommendations exported to Excel successfully!")

Recommendations exported to Excel successfully!
